> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# Pilares da OO

<img src="https://drive.google.com/uc?id=1mKztPpEvd0KE-mFYruZ1J1j-BKzvhBDU" width=600>

* **Encapsulamento**: Expressa-se na prática através do controle de acesso, expondo publicamente apenas os atributos e métodos necessários para uso externo.
* **Herança**: Criação de novas classes baseadas em classes existentes, herdando seus atributos e métodos.
* **Abstração**: Simplificar a complexidade ao ocultar detalhes de implementação de ume entidade ou relação entre entidades. Uma das expressões desse pilar é a criação de _interfaces_.
* **Polimorfismo**: Capacidade de uma entidade de assumir diferentes formas, permitindo o uso de uma interface comum para tipos distintos.


## Encapsulamento

> Em Python, **não há mecanismos para restringir totalmente o acesso a atributos e métodos de uma classe** (no máximo você pode dificultar um pouco). Em vez disso, **utiliza-se convenções de nomenclatura** para indicar níveis de acesso e dar recomendações sobre como os membros devem ser utilizados. Essas convenções ajudam a comunicar aos desenvolvedores quais membros são destinados ao uso interno de uma classe e quais fazem parte da interface pública.

#### ❗❗ Use as convenções de nomenclatura ❗❗
Tomei algumas liberdades na tabela a seguir para relacionar as convenções de nomenclatura aos níveis de permissão de acesso de linguagens que obedecem o paradigma da orientação a objetos.

| Nível de acesso  | Convenção de nomenclatura |
| ------------- | :------------- |
| Público       | Use as convenções conhecidas: `uma_variavel, umaFuncao()`  |
| Privado       | Adicione um underline (`_`) **antes** do nome: `_uma_variavel`, `_umaFuncao()`  |
| Protegido\*    | Adicione dois underlines (`__`) **antes** do nome: `__uma_variavel`, `__umaFuncao()`  |

Toda vez que você encontrar elementos que iniciam com um único underline, a intenção é que eles sejam privados, ou seja, não foram criados para uso externo. Ao criar nossos próprios módulos ou APIs, devemos adotar a mesma padronização. Já a padronização com dois underlines requer uma explicação um pouco mais longa que você verá a seguir.

#### 🌀 *Name mangling* (Embaralhamento de nome)

Embaralhamento de nomes é o recurso que mais se aproxima da ideia de controle de acesso. Ao declarar um membro cujo nome inicia com dois underlines (`__membro`), é aplicada uma transformação automática que adiciona o nome da classe antes do nome do membro (`_NomeDaClasse__membro`). Note que a transformação adota a padronização de membros privados, com um único underline no início do nome.

A intenção aqui é esconder o atributo já que apesar de declarado como `__membro`, qualquer tentativa de acessar `Classe.__membro` lança um erro de execução.


In [ ]:
import json

class Exemplo:
  __mangled_class_var = 0

  def __init__(self):
    self.__mangled_instance_var = ''

  def __mangled_method(self):
    pass

vars(Exemplo)

mappingproxy({'__module__': '__main__',
              '_Exemplo__mangled_class_var': 0,
              '__init__': <function __main__.Exemplo.__init__(self)>,
              '_Exemplo__mangled_method': <function __main__.Exemplo.__mangled_method(self)>,
              '__dict__': <attribute '__dict__' of 'Exemplo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Exemplo' objects>,
              '__doc__': None})

In [ ]:
instancia = Exemplo()
vars(instancia)

{'_Exemplo__mangled_instance_var': ''}

> Note que as ✨ funções mágicas ✨ são outra expressão do uso do caracter underline para indicar funções que não devem ser acessadas externamente. Com a variação (`__dunder__()`) especifica-se funções que sobrescrevem comportamentos nativos da linguagem.

---
### *Getters* e *Setters*
Ainda dentro de encapsulamento, aprende-se na teoria de OO que a criação de *getters* e *setters* é a melhor maneira de manipular os membros de uma classe sem ferir o pilar do encapsulamento. No Python, todos os membros de uma entidade estão expostos, mas a linguagem **permite modificar o comportamento das operações nativas de atribuição e consulta de valor** através das seguintes funções:

* O método a seguir é chamado sempre que tentamos acessar um membro da classe. Se o membro existe, ele é retornado, senão é lançado um `AttributeError`.
```python
__getattribute__(self, key) -> object
```

* Caso o seguinte método exista, ele é chamado quando tentamos acessar membros inexistentes da classe antes que `__getattribute__` lance seu erro. Ou seja, é uma chance de lidar com o acesso a membros inexistentes da classe.
```python
__getattr__(self, key) -> object
```

* Uma característica muito peculiar do Python é a criação de atributos dinamicamente, ou seja, em tempo de execução. Então independente de um atributo existir ou não, podemos definir o método mágico a seguir para alterar o comportamento da atribuição de valor a um atributo.
```python
__setattr__(self, key, value) -> None
```

A estruturação que veremos a seguir permite quaisquer alterações nas operações de consulta e alteração de membros de uma classe, por exemplo
* realizando checagem de tipos
* implementando regras de encapsulamento que proibam o acesso a determinados membros.
* etc...

In [ ]:
class Exemplo:

    def __init__(self,):
        print('Iniciando minha classe!')
        self.num = 0

    def __setattr__(self, att, x):
        print(f'Definindo o valor de {att}')
        object.__setattr__(self, att, x)

    def __getattr__(self, att):
        print(f'{att} não existe na instância!')
        return None

    def __getattribute__(self, att):
        print(f'Consultando o valor de {att}')
        return object.__getattribute__(self, att)

Ao realizar operações de consulta ou atribuição de valores a atributos da classe implementada anteriormente, invocamos os comportamentos customizados por nós, incluindo os prints que nos ajudam a acompanhar o fluxo de execução.


In [ ]:
print('--> obj = Exemplo()')
obj = Exemplo()
print('--> obj.num = 10')
obj.num = 10
print('--> obj.num')
print(obj.num)

--> obj = Exemplo()
Iniciando minha classe!
Definindo o valor de num
--> obj.num = 10
Definindo o valor de num
--> obj.num
Consultando o valor de num
10


O exemplo a seguir apresenta um fluxo de acesso a um atributo inexistente da classe implementada. Note que `__getattribute__` é invocado primeiro, disparando automaticamente a invocação de `__getattr__` ao perceber a inexistência do atributo.


In [ ]:
print('--> obj2 = Exemplo()')
obj2 = Exemplo()
print('--> obj2.variavel')
print(obj2.variavel)

--> obj2 = Exemplo()
Iniciando minha classe!
Definindo o valor de num
--> obj2.variavel
Consultando o valor de variavel
variavel não existe na instância!
None


Ao realizar uma operação de atribuição com um atributo que não existe, ele passa a existir dinamicamente no momento que a instrução é executada.


In [ ]:
print('--> obj2.variavel = ...')
obj2.variavel = 'olá eu sou uma variável'
print('--> obj2.variavel')
print(obj2.variavel)

type(obj), vars(obj), type(obj2), vars(obj2)

--> obj2.variavel = ...
Definindo o valor de variavel
--> obj2.variavel
Consultando o valor de variavel
olá eu sou uma variável
Consultando o valor de __dict__
Consultando o valor de __dict__


(__main__.Exemplo,
 {'num': 10},
 __main__.Exemplo,
 {'num': 0, 'variavel': 'olá eu sou uma variável'})


### `property`

Vale mencionar brevemente outro recurso para controle de atributos de uma classe: as propriedades. Através da função embutida `property()` podemos criar um objeto (de tipo `property`) associando a ele funções de `get`, `set` e `del`, além de uma `docstring` descrevendo sua funcionalidade, como apresentado a seguir:

```python
property(fget=None, fset=None, fdel=None, doc=None) -> property
```


In [ ]:
class Exemplo:
    def __init__(self,):
        self._cont = 0

    def getcont(self):
        self._cont += 1
        return self._cont

    def setcont(self, value):
        raise Exception('Esse valor não pode ser alterado')

    def delcont(self):
        del self._cont

    cont = property(getcont, setcont, delcont,
                    doc="Eu sou uma propriedade de contagem")

A propriedade `cont` define um mecanismo de controle do atributo "privado" `_cont`. De acordo com a implementação, ao consultar o valor de `cont` (via `getcont`), o atributo interno `_cont` é incrementado e seu valor retornado. Já ao tentar atribuir um valor a `cont`, um erro de execução será lançado e nenhuma alteração de valor é realizada. Emulamos assim o controle de acesso ao atributo da instância.

Note que a propriedade `cont` é acessada como um atributo comum, mas internamente invoca os respectivos métodos implementados. Por isso vemos o valor de `ex.cont` crescer a cada novo print.


In [ ]:
for i in range(10):
    print(ex.cont, end= ' ')

ex.cont = 0

1 2 3 4 5 6 7 8 9 10 

Exception: Esse valor não pode ser alterado

Ao criar a propriedade dessa maneira, todos métodos declarados, além da propriedade em si, aparecem no dicionário de atributos da classe.


In [ ]:
ex = Exemplo()
vars(Exemplo), vars(ex)

(mappingproxy({'__module__': '__main__',
               '__init__': <function __main__.Exemplo.__init__(self)>,
               'getcont': <function __main__.Exemplo.getcont(self)>,
               'setcont': <function __main__.Exemplo.setcont(self, value)>,
               'delcont': <function __main__.Exemplo.delcont(self)>,
               'cont': <property at 0x794e5dda5800>,
               '__dict__': <attribute '__dict__' of 'Exemplo' objects>,
               '__weakref__': <attribute '__weakref__' of 'Exemplo' objects>,
               '__doc__': None}),
 {'_cont': 0})

In [ ]:
help(ex)

Help on Exemplo in module __main__ object:

class Exemplo(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  delcont(self)
 |  
 |  getcont(self)
 |  
 |  setcont(self, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  cont
 |      Eu sou uma propriedade de contagem



Um recurso que torna o uso da `property` mais conciso é o ✨ decorador ✨ de mesmo nome.
```python
@property
def nome_da_propriedade(self):
    """docstring"""
    return ...
```
Ao definir uma função decorada com `@property` estamos fazendo três coisas ao mesmo tempo:
* Criando um atributo da classe com o nome dado à função decorada
* Definindo a `docstring` que descreve a propriedade
* Definindo o comportamento da função `get`

Resta apenas duas funcionalidades não implementadas, o *setter* e o *deleter*. Podemos criá-las definindo métodos com os seguintes decoradores:
* O `nome_da_propriedade` definido pode ser usado como um novo decorador. O método logo abaixo deve também se chamar `nome_da_propriedade` e define o comportamento do *setter*.
```python
@nome_da_propriedade.setter
def nome_da_propriedade(self, valor)
```

* Para implementar o *deleter* definir o seguinte método decorado
```python
@nome_da_propriedade.deleter
def nome_da_propriedade(self)
```



In [ ]:
class Exemplo:
    def __init__(self,):
        self._cont = 0

    @property
    def cont(self): # comportamento do get
        """Eu sou uma propriedade de contagem."""
        self._cont += 1
        return self._cont

    @cont.setter
    def cont(self, value):
        raise Exception('Esse valor não pode ser alterado')

    @cont.deleter
    def cont(self): del self._cont

In [ ]:
ex = Exemplo()
vars(Exemplo), vars(ex)

(mappingproxy({'__module__': '__main__',
               '__init__': <function __main__.Exemplo.__init__(self)>,
               'cont': <property at 0x7d1086ee4c20>,
               '__dict__': <attribute '__dict__' of 'Exemplo' objects>,
               '__weakref__': <attribute '__weakref__' of 'Exemplo' objects>,
               '__doc__': None}),
 {'_cont': 0})

In [ ]:
help(ex)

Help on Exemplo in module __main__ object:

class Exemplo(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  cont
 |      Eu sou uma propriedade de contagem.



In [ ]:
for i in range(10):
    print(ex.cont, end= ' ')

ex.cont = 0

1 2 3 4 5 6 7 8 9 10 

Exception: Esse valor não pode ser alterado

Internamente, a `property` implementa o protocolo *descriptor*, criando um objeto descritor que você pode conhecer em detalhes [na documentação](https://docs.python.org/3/howto/descriptor.html) ou [nesse tutorial aqui](https://realpython.com/python-classes/#property-and-descriptor-based-attributes). O problema desse recurso é não se adequar bem a casos de herança. Você pode ler mais sobre isso nesse [tutorial do Real Python](https://realpython.com/python-getter-setter/#using-inheritance-getter-and-setters-vs-properties).

## Referências

* https://realpython.com/python-classes/#public-vs-non-public-members
* https://peps.python.org/pep-0008/#method-names-and-instance-variables
* https://docs.python.org/3/howto/descriptor.html
* https://realpython.com/python-getter-setter/

---

## Herança

Na herança, uma nova classe (subclasse) é criada com base em uma classe existente (superclasse). Isso permite que a subclasse herde os métodos e atributos da superclasse, promovendo a reutilização de código e a criação de hierarquias lógicas entre as classes.

A relação hierárquica entre classes pode ser representada de maneira abstrata como um diagrama de classes UML. Na representação a seguir a `SubClasse` define seus próprios atributos e métodos mas implícitamente possui instâncias dos atributos e métodos da `SuperClasse`.
```
+---------------------+
|  SuperClasse        |
| - atributos_super   |
| - métodos_super     |
+---------------------+
          ^        
          |
          |
 +------------------+
 |   SubClasse      |
 | - atributos_sub  |
 | - métodos_sub    |
 +------------------+
```

A sintaxe do Python que define essa relação é apresentada a seguir. Não existe uma palavra reservada para apontar a relação de herança. Basta passar o nome da superclasse na definição da subclasse, usando a sintaxe de parênteses.
```python
class SuperClasse:
    # Defina métodos e atributos da superclasse
    # ...
    
# SubClasse que herda de SuperClasse
class SubClasse(SuperClasse):
    # Defina métodos e atributos específicos da SubClasse
    # ...
```

❗❗ **Importante** ❗❗

Em Python, todas as classes criadas herdam implicitamente da classe genérica `object`, que é a superclasse base da linguagem. Isso significa que a hierarquia vai existir mesmo que você não especifique explicitamente uma superclasse ao definir uma nova classe. Essa herança implícita fornece à nova classe uma série de métodos e comportamentos padrão, como os métodos mágicos `__init__`, `__str__`, `__repr__`, entre outros, que você já aprendeu que podem ser usados ou sobrescritos!

O comando `help(Exemplo)`, executado a seguir, mostra que a classe `Exemplo` é uma subclasse de `object`, mesmo que isso não tenha sido explicitamente declarado na definição de `Exemplo`.

```python
class Exemplo(builtins.object)
```

In [ ]:
help(Exemplo)

Help on class Exemplo in module __main__:

class Exemplo(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  cont
 |      Eu sou uma propriedade de contagem.



### Vilões de Novela 🦹

Para conhecer as particularidades da relação de herança no Python, vejamos um exemplo prático! No exemplo, a classe `Antagonista` é a superclasse representando vilões e vilãs de novelas brasileiras, com atributos `nome` e `bordao` e o método `dizer_bordao` (que conhecendo novelas será muito chamado! 😆).

Qualquer antagonista genérico pode ser instanciado a partir dessa classe. Vai ser o caso da vilã Norma Gusmão com o seu bordão "*Eu sou rica!*".  Mas nesse exemplo queremos implementar classes especializadas para antagonistas de destaque: Nazaré Tedesco (Nazaré confusa) e Felix (o rei dos bordões)!

<img src="https://upload.wikimedia.org/wikipedia/pt/c/cd/Nazar%C3%A9_Confusa.jpg" height=200>
<img src="https://conteudo.imguol.com.br/c/entretenimento/2013/12/17/felix-vilao-interpretado-por-mateus-solano-na-novela-amor-a-vida-da-globo-1387313146611_300x200.jpg" height=200>
<img src="https://www.fapesb.ba.gov.br/wp-content/uploads/2016/03/rica.png" height=200>

Aqui está o diagrama de classes UML do nosso belo exemplo:

```
                 +------------------+
                 |  Antagonista     |
                 +------------------+
                 | - nome           |
                 | - bordao         |
                 +------------------+
                 | + dizer_bordao() |
                 +------------------+
                         ^
        _________________|________________
       |                                  |
+---------------------+        +--------------------+
|  NazareTedesco      |        |  Felix             |
+---------------------+        +--------------------+
| - isConfusa         |        | - apelidos         |
+---------------------+        +--------------------+
| + empurrar_escada() |        | + gerar_apelido()  |
+---------------------+        +--------------------+
```

Leia a implementação da célula a seguir, procurando os seguinte elementos **nas subclasses**:
* Note que a assinatura do construtor das subclasses recebe os atributos tanto da superclasse, quanto seus próprios atributos:
```python
# Classe NazareTedesco
# atributos da superclasse: nome, bordao
# atributo da subclasse: isConfusa
__init__(self, nome, bordao, isConfusa=True):
```

* O **primeiro passo** do construtor é inicializar a superclasse, antes de seguir para suas especificidades. Isso é possível recuperando uma **referência à superclasse** através da chamada de **`super()`**. A partir de seu retorno invocamos explicitamente o método mágico `__init__()`, ou seja, o construtor da super classe.
```python
super().__init__(nome, bordao)
```

> ❗ O `super()` é muito poderoso e flexível para lidar com as mais variadas relações de herança. Ele resolve casos de heranças múltiplas, ou hierarquias de mais de 2 níveis. Caso deseje conhecer o seu poder total (é mais de 8000!), recomendo ler esse tutorial do Real Python: [Supercharge Your Classes With Python super()](https://realpython.com/python-super/). *Lembre-se que você pode traduzir para português no navegador*.


In [ ]:
import random
class Antagonista():
    def __init__(self, nome, bordao):
        self.nome = nome
        self.bordao = bordao

    def dizer_bordao(self):
        print(f"{self.nome} diz: {self.bordao}.")


class NazareTedesco(Antagonista):
    def __init__(self, nome, bordao, isConfusa=True):
        super().__init__(nome, bordao)
        self.isConfusa = isConfusa

    def empurrar_escada(self):
        print(f"{self.nome} está te empurrando da escada!! 😱")


class Felix(Antagonista):
    def __init__(self, nome, bordao):
        super().__init__(nome, bordao)
        self.apelidos = {'substantivos': ['lacraia', 'bofe', 'rainha', 'criatura'],
                         'adjetivos': ['do olho colorido', 'de ouro', 'do hot dog']}

    def gerar_apelido(self):
        ids = random.randrange(len(self.apelidos['substantivos']))
        ida = random.randrange(len(self.apelidos['adjetivos']))

        return self.apelidos['substantivos'][ids] + self.apelidos['adjetivos'][ida]


norma  = Antagonista("Norma Gusmão", "Eu sou rica!!!!")
nazare = NazareTedesco("Nazaré Tedesco", "O tempo só te valoriza", True)
felix  = Felix("Félix", "Eu salguei a santa ceia, só pode!")

vars(norma), vars(nazare), vars(felix)

({'nome': 'Norma Gusmão', 'bordao': 'Eu sou rica!!!!'},
 {'nome': 'Nazaré Tedesco',
  'bordao': 'O tempo só te valoriza',
  'isConfusa': True},
 {'nome': 'Félix',
  'bordao': 'Eu salguei a santa ceia, só pode!',
  'apelidos': {'substantivos': ['lacraia', 'bofe', 'rainha', 'criatura'],
   'adjetivos': ['do olho colorido', 'de ouro', 'do hot dog']}})

Quando você usa o comando `help(NazareTedesco)`, por exemplo, o Python te dá uma visão geral da classe, mostrando como ela se relaciona com as outras classes na hierarquia de herança. Vamos destacar alguns aspectos:
* ***Method resolution order*** (MRO): A ordem em que Python vai procurar métodos ou atributos se você tentar acessá-los numa instância de NazareTedesco. Nesse caso, o Python primeiro olha na própria NazareTedesco. Se ele não encontrar o que precisa lá, ele sobe a hierarquia e olha na superclasse Antagonista. Se ainda assim não encontrar, vai para a classe base `object`.

* **Métodos definidos aqui**: Logo depois, o help mostra os métodos que são especificamente definidos na classe NazareTedesco. Aqui, você vê o construtor `__init__`, e o método `empurrar_escada`, que é exclusivo da Nazaré e não tem nada a ver com a superclasse Antagonista.

* **Métodos herdados de Antagonista**: Por último, o help lista os métodos que `NazareTedesco` herdou da sua superclasse  `Antagonista`. O único método herdado é o `dizer_bordao`.

In [ ]:
help(NazareTedesco)

Help on class NazareTedesco in module __main__:

class NazareTedesco(Antagonista)
 |  NazareTedesco(nome, bordao, isConfusa=True)
 |  
 |  Method resolution order:
 |      NazareTedesco
 |      Antagonista
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, nome, bordao, isConfusa=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  empurrar_escada(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Antagonista:
 |  
 |  dizer_bordao(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Antagonista:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### Sobrescrita de métodos

Vamos alterar a classe `Felix` para sobrescrever o método `dizer_bordao` que foi herdado da superclasse. Afinal o Felix é o rei dos bordões e precisa da sua própria implementação.



In [ ]:
class Felix(Antagonista):
    def __init__(self, nome, bordoes):
        super().__init__(nome, bordoes)
        self.apelidos = {'substantivos': ['lacraia', 'bofe', 'rainha', 'criatura'],
                         'adjetivos': ['do olho colorido', 'de ouro', 'do hot dog']}

    # método da superclasse sobrescrito na subclasse
    def dizer_bordao(self):
        idx = random.randrange(len(self.bordao))
        print(f"{self.nome} diz: {self.bordao[idx]}.")

    def gerar_apelido(self):
        ids = random.randrange(len(self.apelidos['substantivos']))
        ida = random.randrange(len(self.apelidos['adjetivos']))

        return self.apelidos['substantivos'][ids] + self.apelidos['adjetivos'][ida]



bordoes = ['Eu salguei a santa ceia, só pode!',
           'Lavei cueca na manjedoura',
           'Piquei salsinha na tábua dos 10 mandamentos',
           'Cobrei juros das 30 moedas de Judas']

felix = Felix('Félix', bordoes)

Algumas coisas interessantes acontecem na estrutura da classe e na saída do comando `help(Felix)`

* Na versão original da classe `Felix`, o método `dizer_bordao` aparecia como "Métodos herdados de Antagonista". Agora que o Felix tem sua própria implementação (sobrescrita), esse trecho do `help` nem existe mais! Felix não herda mais nenhum método da superclasse, apenas seus atributos.

* Consequentemente, `dizer_bordao` agora aparece em "Métodos definidos aqui". Note também que o comentário adicionado logo acima do método aparece automaticamente como uma documentação (que lindo 🥹, comente seu código!).

* A ordem de resolução dos métodos não muda, pois a estrutura de herança permanece a mesma.





In [ ]:
help(felix)

Help on Felix in module __main__ object:

class Felix(Antagonista)
 |  Felix(nome, bordoes)
 |  
 |  Method resolution order:
 |      Felix
 |      Antagonista
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, nome, bordoes)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  dizer_bordao(self)
 |      # método da superclasse sobrescrito na subclasse
 |  
 |  gerar_apelido(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Antagonista:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### Herança Múltipla

Exemplo a seguir adaptado do [melhor e mais completo tutorial do Real Python](https://realpython.com/python-classes/#multiple-inheritance) sobre orientação a objetos no Python.

Vamos implementar a abstração de um carro voador!

<img width=500 src="https://images.squarespace-cdn.com/content/v1/5e1e8a13e8a3102d5c82d1b1/1606512300229-DRPYFBISPBE1JUUZL92Z/Jetsons.jpeg">


Nesse cenário, temos a definição de uma classe `Vehicle` (veículo), que será a base de todas classes. No segundo nível da hierarquia temos `Car` (carro) e `Aircraft` (aeronave), ambos herdando de veículo. Temos por fim um terceiro nível hierárquico que define `FlyingCar` (carro voador), com uma relação de **herança múltipla**, herdando simultaneamente de carro e de aeronave. Veja a representação UML a seguir.


```
      +--------------------------------+
      |             Vehicle            |
      +--------------------------------+
      | - make: str                    |
      | - model: str                   |
      | - color: str                   |
      +--------------------------------+
      | + __init__(make, model, color) |
      | + start()                      |
      | + stop()                       |
      | + show_technical_specs()       |
      +--------------------------------+
                       ^
          _____________|_____________
         |                           |
+-----------------+         +-----------------+   
|       Car       |         |     Aircraft    |
+-----------------+         +-----------------+
+-----------------+         +-----------------+
| + drive()       |         | + fly()         |
| + stop()        |         | + stop()        |
+-----------------+         +-----------------+
         ^                           ^
         |                           |
          ---------------------------
                       |
              +-----------------+
              |     FlyingCar   |
              +-----------------+
              +-----------------+
              +-----------------+
```







A pergunta que fica é:

> ❓ Se todo mundo na hierarquia tem o método `stop`, o que acontece se eu defino um carro voador e invoco este método ❓
```python
carro_jetsons = FlyingCar('Spacely Space Sprockets Inc.', 'Flying car', 'green')
carro_jetsons.stop() # qual stop estou chamando?
```

Vamos implementar essa relação de maneira prática, só com alguns prints pra saber onde estamos na execução.

In [ ]:
class Vehicle:
    def __init__(self, make, model, color):
        self.make = make
        self.model = model
        self.color = color

    def start(self):
        print("Ligando o motor...")

    def stop(self):
        print("Desligando o motor...")

    def show_technical_specs(self):
        print(f"Marca: {self.make}")
        print(f"Modelo: {self.model}")
        print(f"Cor: {self.color}")

class Car(Vehicle):
    def drive(self):
        print("Dirigindo na rua...")

    def stop(self):
        print("Eu sou um carro e vou desligar o motor hein?!")

class Aircraft(Vehicle):
    def fly(self):
        print("Voando no céu...")

    def stop(self):
        print("Não desligue o motor no ar!!!")

class FlyingCar(Car, Aircraft):
    pass

#### 💡 Method Resolution Order (MRO) 💡

De acordo com a ordem de resolução da classe `FlyingCar`, ao invocar um método, o interpretador vai procurar este método primeiro na própria classe (sempre), e em seguida buscará na classe `Car`, a primeira definida na relação de herança múltipla.

```python
class FlyingCar(Car, Aircraft):
```

In [ ]:
FlyingCar.__mro__

(__main__.FlyingCar, __main__.Car, __main__.Aircraft, __main__.Vehicle, object)

❗ Que perigo ❗

Dessa maneira, seu carro voador vai simplesmente desligar o motor! Experimente alterar a definição de `FlyingCar` trocando a ordem da herança e veja como a ordem de resolução vai ser alterada.

```python
class FlyingCar(Aircraft, Car):
```

Por fim execute o código a seguir e veja como isso acontecerá na prática.

In [ ]:
carro_jetsons = FlyingCar('Spacely Space Sprockets Inc.', 'Flying car', 'green')
carro_jetsons.stop()

Eu sou um carro e vou desligar o motor hein?!


In [ ]:
help(FlyingCar)

Help on class FlyingCar in module __main__:

class FlyingCar(Car, Aircraft)
 |  FlyingCar(make, model, color)
 |  
 |  Method resolution order:
 |      FlyingCar
 |      Car
 |      Aircraft
 |      Vehicle
 |      builtins.object
 |  
 |  Methods inherited from Car:
 |  
 |  drive(self)
 |  
 |  stop(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Aircraft:
 |  
 |  fly(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Vehicle:
 |  
 |  __init__(self, make, model, color)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  show_technical_specs(self)
 |  
 |  start(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Vehicle:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (i